In [1]:
#!/usr/bin/env python3
"""
s1_multivariate_pass_model.py

Goal:
- Demonstrate that S1 metrics (especially S1_Graph_overall_score / connectivity)
  remain predictive of pass/fail AFTER controlling for:
    - pipeline complexity (nodes, edges, depth, components)
    - orchestrator
    - Prompt2DAG strategy (Template/LLM/Hybrid) [Method column]
    - base LLM (Std_LLM)

Outputs:
- Logistic regression odds ratios + 95% CI (statsmodels)
- AUC (sklearn) for baseline vs full model
- Optional GroupKFold AUC by Pipeline_ID (recommended)
"""

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, GroupKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

import statsmodels.formula.api as smf

# -----------------------
# Config
# -----------------------
CSV_PATH = "/Users/abubakarialidu/Desktop/Data Result/all_sessions_cleaned.csv"

FILTER_PROMPT2DAG_ONLY = True  # S1 metrics exist there
TEST_SIZE = 0.25
RANDOM_STATE = 42

# Candidate column names for the "base model" dimension
STD_LLM_CANDIDATES = ["Std_LLM", "STD_LLM", "Base_LLM", "Model", "LLM"]

# Complexity features
COMPLEXITY_COLS = [
    "S1_Graph_total_nodes_in_flow",
    "S1_Graph_total_edges",
    "S1_Graph_total_components",
    "S1_Graph_max_pipeline_depth",
]

# S1 features (keep modest to reduce multicollinearity)
S1_FEATURES = [
    "S1_Graph_overall_score",
    "S1_Graph_Node_Connectivity_score",
    "S1_Sem_ROUGE1_norm",
    "S1_Sem_KeyTerm_rate",
]

# Categorical controls
CATEGORICAL_COLS = ["Orchestrator", "Method"]  # Method = Template/LLM/Hybrid for Prompt2DAG

# Group leakage control
GROUP_COL = "Pipeline_ID"  # use GroupKFold if available

# -----------------------
# Load
# -----------------------
df = pd.read_csv(CSV_PATH)
print(f"Loaded {len(df):,} rows, {len(df.columns)} cols")

# Method classification (if you haven't already stored Method in the CSV)
def classify_method(row):
    workflow = row.get("Workflow", "")
    strategy = str(row.get("Strategy") or "").lower()
    if workflow == "Prompt2DAG":
        if "template" in strategy:
            return "Prompt2DAG (Template)"
        elif "llm" in strategy:
            return "Prompt2DAG (LLM)"
        elif "hybrid" in strategy:
            return "Prompt2DAG (Hybrid)"
        else:
            return f"Prompt2DAG ({row.get('Strategy', 'Unknown')})"
    return workflow

if "Method" not in df.columns:
    df["Method"] = df.apply(classify_method, axis=1)

if FILTER_PROMPT2DAG_ONLY:
    df = df[df["Workflow"] == "Prompt2DAG"].copy()
    print(f"Filtered to Prompt2DAG: {len(df):,} rows")

# Passed numeric
df["Passed_num"] = df["Passed"].astype(int)

# Find Std_LLM column if present
std_llm_col = None
for c in STD_LLM_CANDIDATES:
    if c in df.columns:
        std_llm_col = c
        break

if std_llm_col is not None:
    print(f"Using base LLM column: {std_llm_col}")
    if std_llm_col not in CATEGORICAL_COLS:
        CATEGORICAL_COLS = CATEGORICAL_COLS + [std_llm_col]
else:
    print("WARNING: No Std_LLM column found. Model will not control for base LLM.")

# Ensure required columns exist
needed_numeric = [c for c in (COMPLEXITY_COLS + S1_FEATURES) if c in df.columns]
missing_numeric = [c for c in (COMPLEXITY_COLS + S1_FEATURES) if c not in df.columns]
if missing_numeric:
    print("WARNING: Missing some numeric predictors:", missing_numeric)

needed_cats = [c for c in CATEGORICAL_COLS if c in df.columns]
missing_cats = [c for c in CATEGORICAL_COLS if c not in df.columns]
if missing_cats:
    print("WARNING: Missing some categorical predictors:", missing_cats)

# Drop rows with missing predictors (simple approach)
model_cols = ["Passed_num"] + needed_numeric + needed_cats
if GROUP_COL in df.columns:
    model_cols += [GROUP_COL]

df_m = df[model_cols].dropna().copy()
print(f"Modeling rows after dropna: {len(df_m):,}")

# -----------------------
# Build Baseline vs Full features
# -----------------------
baseline_numeric = [c for c in COMPLEXITY_COLS if c in df_m.columns]
full_numeric = baseline_numeric + [c for c in S1_FEATURES if c in df_m.columns]

categorical = needed_cats

print("\nBaseline numeric:", baseline_numeric)
print("Full numeric:", full_numeric)
print("Categorical:", categorical)

# -----------------------
# (1) Statsmodels: Odds ratios + CI (Full model)
# -----------------------
print("\n" + "=" * 110)
print("STATSMODELS LOGISTIC REGRESSION (FULL MODEL): Odds Ratios + 95% CI")
print("=" * 110)

# Build formula: Passed_num ~ num + C(cat) ...
formula_terms = []
formula_terms += full_numeric
formula_terms += [f"C({c})" for c in categorical]

formula = "Passed_num ~ " + " + ".join(formula_terms)
print("\nFormula:\n", formula)

# Fit model
logit = smf.logit(formula=formula, data=df_m).fit(disp=False)

# OR table
params = logit.params
conf = logit.conf_int()
or_table = pd.DataFrame({
    "coef": params,
    "OR": np.exp(params),
    "CI_low": np.exp(conf[0]),
    "CI_high": np.exp(conf[1]),
    "p": logit.pvalues
}).sort_values("p")

print("\nTop predictors (sorted by p-value):")
print(or_table.head(25).to_string(float_format=lambda x: f"{x:0.4f}"))

# Highlight key S1 effects if present
for key in ["S1_Graph_overall_score", "S1_Graph_Node_Connectivity_score"]:
    if key in or_table.index:
        r = or_table.loc[key]
        print(f"\nKEY EFFECT: {key}")
        print(f"  OR = {r['OR']:.3f}  (95% CI {r['CI_low']:.3f}–{r['CI_high']:.3f}), p={r['p']:.3g}")

# -----------------------
# (2) Sklearn: AUC baseline vs full (hold-out split)
# -----------------------
print("\n" + "=" * 110)
print("SKLEARN AUC (Hold-out): Baseline vs Full")
print("=" * 110)

X = df_m[baseline_numeric + full_numeric].copy()  # will be overwritten below
y = df_m["Passed_num"].values

# ColumnTransformer: scale numeric, one-hot categorical
def make_pipeline(numeric_cols, categorical_cols):
    pre = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ],
        remainder="drop"
    )
    clf = LogisticRegression(max_iter=500, solver="liblinear")
    return Pipeline([("pre", pre), ("clf", clf)])

# Split
X_train, X_test, y_train, y_test = train_test_split(
    df_m, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

# Baseline model
pipe_base = make_pipeline(baseline_numeric, categorical)
pipe_base.fit(X_train[baseline_numeric + categorical], y_train)
p_base = pipe_base.predict_proba(X_test[baseline_numeric + categorical])[:, 1]
auc_base = roc_auc_score(y_test, p_base)

# Full model
pipe_full = make_pipeline(full_numeric, categorical)
pipe_full.fit(X_train[full_numeric + categorical], y_train)
p_full = pipe_full.predict_proba(X_test[full_numeric + categorical])[:, 1]
auc_full = roc_auc_score(y_test, p_full)

print(f"Baseline AUC (complexity + controls): {auc_base:.4f}")
print(f"Full AUC (+ S1 metrics):             {auc_full:.4f}")
print(f"Δ AUC:                               {auc_full - auc_base:+.4f}")

# -----------------------
# (3) Optional: GroupKFold AUC by Pipeline_ID (stronger evidence)
# -----------------------
if GROUP_COL in df_m.columns:
    print("\n" + "=" * 110)
    print("GROUPED CV (GroupKFold by Pipeline_ID): AUC Baseline vs Full")
    print("=" * 110)

    groups = df_m[GROUP_COL].values
    gkf = GroupKFold(n_splits=min(5, df_m[GROUP_COL].nunique()))

    # Need X and y in same df order
    X_base = df_m[baseline_numeric + categorical]
    X_full = df_m[full_numeric + categorical]
    y = df_m["Passed_num"].values

    # Cross-val scores
    aucs_base = cross_val_score(pipe_base, X_base, y, cv=gkf, groups=groups, scoring="roc_auc")
    aucs_full = cross_val_score(pipe_full, X_full, y, cv=gkf, groups=groups, scoring="roc_auc")

    print(f"Baseline AUC (GroupKFold) mean±sd: {aucs_base.mean():.4f} ± {aucs_base.std():.4f}")
    print(f"Full AUC (GroupKFold) mean±sd:     {aucs_full.mean():.4f} ± {aucs_full.std():.4f}")
    print(f"Δ AUC (mean):                      {aucs_full.mean() - aucs_base.mean():+.4f}")
else:
    print("\nNOTE: Pipeline_ID not available; skipping GroupKFold. (Recommended if you have it.)")

print("\nDone.")

Loaded 8,742 rows, 94 cols
Filtered to Prompt2DAG: 5,664 rows
Using base LLM column: Std_LLM
Modeling rows after dropna: 5,664

Baseline numeric: ['S1_Graph_total_nodes_in_flow', 'S1_Graph_total_edges', 'S1_Graph_total_components', 'S1_Graph_max_pipeline_depth']
Full numeric: ['S1_Graph_total_nodes_in_flow', 'S1_Graph_total_edges', 'S1_Graph_total_components', 'S1_Graph_max_pipeline_depth', 'S1_Graph_overall_score', 'S1_Graph_Node_Connectivity_score', 'S1_Sem_ROUGE1_norm', 'S1_Sem_KeyTerm_rate']
Categorical: ['Orchestrator', 'Method', 'Std_LLM']

STATSMODELS LOGISTIC REGRESSION (FULL MODEL): Odds Ratios + 95% CI

Formula:
 Passed_num ~ S1_Graph_total_nodes_in_flow + S1_Graph_total_edges + S1_Graph_total_components + S1_Graph_max_pipeline_depth + S1_Graph_overall_score + S1_Graph_Node_Connectivity_score + S1_Sem_ROUGE1_norm + S1_Sem_KeyTerm_rate + C(Orchestrator) + C(Method) + C(Std_LLM)

Top predictors (sorted by p-value):
                                          coef      OR  CI_low 